# Elephant Detection and GPS Localization Demo 🐘📍

Welcome! This Colab notebook allows you to upload a single drone image, detect elephants within it using a YOLO model, and estimate their GPS coordinates.

**How to use:**
1.  Run the "Setup" cell to install necessary libraries and download the model.
2.  Run the "Upload Image" cell and select a JPG image from your computer.
3.  Run the "Process Image and Get Results" cell. This will perform detection, calculate GPS, and display the results.
4.  The results (image with IDs, coordinates CSV, distances CSV) will be displayed and made available for download.

--- 
## 1. Setup Environment

In [ ]:
# Install necessary packages
!pip install ultralytics piexif geopy pyproj -q
!apt-get install -y exiftool -qq

# Download the YOLO model file (best_xl.pt)
# IMPORTANT: Replace 'YOUR_DOWNLOAD_LINK_HERE' with a direct download link to your 'best_xl.pt' file.
# You can host it on GitHub, Google Drive (get a shareable link & use gdown), or another hosting service.
# Example using gdown (if hosted on Google Drive):
# !pip install gdown -q
# !gdown --id YOUR_FILE_ID -O best_xl.pt
# Example using wget (if hosted elsewhere):
# !wget YOUR_DOWNLOAD_LINK_HERE -O best_xl.pt

# --- Placeholder: Manually upload 'best_xl.pt' if no link is provided --- 
print("Please ensure 'best_xl.pt' is available.")
print("If you haven't provided a download link above, you'll need to upload 'best_xl.pt' manually in the next step or modify this cell to download it.")

# --- If you need to upload the model manually, run the cell below --- 
# from google.colab import files
# print("Please upload 'best_xl.pt':")
# uploaded_model = files.upload()
# if 'best_xl.pt' not in uploaded_model:
#   print("\nERROR: 'best_xl.pt' was not uploaded. Please upload the model file to proceed.")
# else:
#   print("\nModel 'best_xl.pt' uploaded successfully!")

# --- For this demo, we'll try to download from a hypothetical location. --- 
# --- YOU MUST REPLACE THIS or ensure the file is uploaded. --- 
try:
  # Replace with your actual download command
  !wget https://storage.googleapis.com/example-models-public/best_xl.pt -O best_xl.pt
  print("Attempted to download 'best_xl.pt'. Check if successful.")
except Exception as e:
  print(f"Could not download 'best_xl.pt'. Please upload it manually or fix the download link. Error: {e}")

# Import necessary libraries
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import pandas as pd
import time
import os
from pathlib import Path
import matplotlib.pyplot as plt
from pyproj import Transformer
import math
import re
from datetime import datetime, timedelta
import subprocess
import json
import csv
from google.colab import files
from PIL import Image
import io
from geopy.distance import geodesic
from IPython.display import Image as IPImage, display

# Create output directories
os.makedirs("Detections", exist_ok=True)
os.makedirs("Processed_Output", exist_ok=True)

print("\nSetup Complete!")

--- 
## 2. Upload Your Image

In [ ]:
from google.colab import files
import io
from PIL import Image

# Upload image
print("Please upload a single JPG image:")
uploaded = files.upload()

# Load and save the uploaded image
uploaded_image_path = None
for fn in uploaded.keys():
    if fn.lower().endswith('.jpg') or fn.lower().endswith('.jpeg'):
        uploaded_image_path = f"/content/{fn}"
        with open(uploaded_image_path, 'wb') as f:
            f.write(uploaded[fn])
        print(f"\nLoaded and saved image: {fn}")
        
        # Display the uploaded image
        img_display = Image.open(io.BytesIO(uploaded[fn]))
        img_display.thumbnail((400, 400)) # Resize for display
        display(img_display)
        break # Process only the first uploaded JPG

if not uploaded_image_path:
    print("\nERROR: No JPG image was uploaded. Please run this cell again and upload a valid image.")

--- 
## 3. Process Image and Get Results

In [ ]:
# --- Define Helper Functions --- 

def Find_Elephants(image_path):
    # Check if model file exists
    model_path = "best_xl.pt"
    if not os.path.exists(model_path):
        print(f"ERROR: Model file '{model_path}' not found. Please ensure it was downloaded or uploaded correctly in Step 1.")
        return None, None

    model = YOLO(model_path)
    results = model.predict(source=image_path, conf=0.6)
    output_dir = "Detections"
    im1 = cv2.imread(image_path)
    height, width, _ = im1.shape
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    yolo_format_data = []
    num_objects = 0

    for box in results[0].boxes:
        num_objects += 1
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(float, box.xyxy[0])
        cv2.rectangle(im1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(im1, f'Class: {cls}, Conf: {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        xmin_norm = x1 / width
        ymin_norm = y1 / height
        xmax_norm = x2 / width
        ymax_norm = y2 / height
        yolo_format_data.append(f"{cls} {conf:.6f} {xmin_norm:.6f} {ymin_norm:.6f} {xmax_norm:.6f} {ymax_norm:.6f}")

    output_image_path = os.path.join(output_dir, f"{base_name}_detection.jpg")
    output_txt_path = os.path.join(output_dir, f"{base_name}.txt")
    cv2.imwrite(output_image_path, im1)
    with open(output_txt_path, 'w') as f:
        f.write("\n".join(yolo_format_data))
    print(f"Detection complete. Found {num_objects} objects. Saved detection image and data.")
    return output_txt_path, output_image_path

def extract_exiftool_metadata(image_path):
    result = subprocess.run(['exiftool', '-j', image_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"exiftool failed: {result.stderr}")
    metadata = json.loads(result.stdout)[0]
    return metadata

def dms_to_decimal(dms_str):
    pattern = r'(\d+)[^\d]+(\d+)[^\d]+([\d.]+)[^\d]+([NSEW])'
    match = re.match(pattern, dms_str.strip())
    if not match: raise ValueError(f"Invalid DMS format: {dms_str}")
    degrees, minutes, seconds, direction = match.groups()
    decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
    if direction in ['S', 'W']: decimal = -decimal
    return decimal

def extract_metadata(image_path):
    metadata = extract_exiftool_metadata(image_path)
    rel_alt = float(metadata.get('RelativeAltitude', 0))
    pitch_deg = float(metadata.get('GimbalPitchDegree', metadata.get('GimbalPitch', -90)))
    focal_length_str = metadata.get('FocalLength', '0')
    focal_length_mm = float(focal_length_str.split(' ')[0])
    sensor_width_mm, sensor_height_mm = 17.3, 13.0
    img_width, img_height = int(metadata.get('ImageWidth', 0)), int(metadata.get('ImageHeight', 0))
    gps_lat, gps_lon = metadata.get('GPSLatitude'), metadata.get('GPSLongitude')
    if not gps_lat or not gps_lon:
       raise ValueError("GPSLatitude or GPSLongitude not found in EXIF data.")
    latitude, longitude = dms_to_decimal(gps_lat), dms_to_decimal(gps_lon)
    return latitude, longitude, rel_alt, pitch_deg, focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height

def get_camera_intrinsics(f_mm, s_w_mm, s_h_mm, i_w, i_h):
    fx = i_w * f_mm / s_w_mm
    fy = i_h * f_mm / s_h_mm
    cx, cy = i_w / 2, i_h / 2
    return np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

def get_rotation_matrix(pitch_deg, yaw_deg=0.0, roll_deg=0.0):
    p, y, r = map(math.radians, [pitch_deg, yaw_deg, roll_deg])
    R_x = np.array([[1, 0, 0], [0, math.cos(r), -math.sin(r)], [0, math.sin(r), math.cos(r)]])
    R_y = np.array([[math.cos(p), 0, math.sin(p)], [0, 1, 0], [-math.sin(p), 0, math.cos(p)]])
    R_z = np.array([[math.cos(y), -math.sin(y), 0], [math.sin(y), math.cos(y), 0], [0, 0, 1]])
    return R_z @ R_y @ R_x

def enu_to_wgs84(x, y, origin_lat, origin_lon):
    t_local = Transformer.from_crs("epsg:4326", "epsg:4978", always_xy=True)
    t_wgs = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)
    ecef_x0, ecef_y0, ecef_z0 = t_local.transform(origin_lon, origin_lat, 0)
    # Note: This is a simplified transformation assuming ENU aligns with ECEF tangent plane
    # For higher accuracy, a full ENU to ECEF transformation is needed.
    # Using a simplified approach here by adding to ECEF x,y
    ecef_x = ecef_x0 + x
    ecef_y = ecef_y0 + y
    ecef_z = ecef_z0
    lon, lat, _ = t_wgs.transform(ecef_x, ecef_y, ecef_z)
    return lat, lon

def image_point_to_gps(u, v, K, R, cam_h, o_lat, o_lon):
    pixel = np.array([u, v, 1])
    ray_cam = np.linalg.inv(K) @ pixel
    ray_world = R @ ray_cam
    if ray_world[2] >= 0: return None, None # Avoid points above horizon
    scale = -cam_h / ray_world[2]
    ground_point = ray_world * scale
    enu_x, enu_y = ground_point[0], ground_point[1]
    return enu_to_wgs84(enu_x, enu_y, o_lat, o_lon)

def run_pipeline(image_path, bounding_boxes):
    try:
        lat, lon, alt, pitch, f_mm, s_w, s_h, i_w, i_h = extract_metadata(image_path)
    except Exception as e:
        print(f"Error extracting metadata: {e}")
        return None

    K = get_camera_intrinsics(f_mm, s_w, s_h, i_w, i_h)
    R = get_rotation_matrix(pitch)
    gps_results = []
    for i, box in enumerate(bounding_boxes, start=1):
        x_min, y_min, x_max, y_max = box
        center_u = (x_min + x_max) / 2 * i_w
        center_v = (y_min + y_max) / 2 * i_h
        gps_lat, gps_lon = image_point_to_gps(center_u, center_v, K, R, alt, lat, lon)
        if gps_lat is not None:
          gps_results.append((i, (center_u, center_v), (gps_lat, gps_lon)))
    return gps_results

def load_bounding_boxes_from_txt(txt_path):
    boxes = []
    if not os.path.exists(txt_path):
        print(f"Warning: Txt file not found at {txt_path}")
        return boxes
    with open(txt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 6:
                _, _, xmin, ymin, xmax, ymax = map(float, parts)
                boxes.append((xmin, ymin, xmax, ymax))
    return boxes

def save_and_display_results(image_path, results, base_name):
    img = cv2.imread(image_path)
    output_dir = "Processed_Output"
    csv_path = os.path.join(output_dir, f"{base_name}_coordinates.csv")
    dist_csv_path = os.path.join(output_dir, f"{base_name}_distances.csv")
    img_path = os.path.join(output_dir, f"{base_name}_processed.jpg")

    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['ObjectID', 'Latitude', 'Longitude'])
        for obj_id, (_, _), (lat, lon) in results:
            writer.writerow([obj_id, f"{lat:.6f}", f"{lon:.6f}"])

    ids = [r[0] for r in results]
    coords = [(r[2][0], r[2][1]) for r in results]
    with open(dist_csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['ObjectID_1', 'ObjectID_2', 'Distance_meters'])
        for i in range(len(results)):
            for j in range(i + 1, len(results)):
                dist = geodesic(coords[i], coords[j]).meters
                writer.writerow([ids[i], ids[j], f"{dist:.2f}"])

    for obj_id, (u, v), (lat, lon) in results:
        pos = (int(u), int(v))
        cv2.circle(img, pos, 10, (0, 0, 255), -1) # Draw red dot
        cv2.putText(img, f"ID {obj_id}", (pos[0] + 15, pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(img, f"{lat:.5f}, {lon:.5f}", (pos[0] + 15, pos[1] + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1, cv2.LINE_AA)

    cv2.imwrite(img_path, img)

    print("\n--- Results ---")
    print(f"Processed image saved to: {img_path}")
    print(f"Coordinates saved to: {csv_path}")
    print(f"Distances saved to: {dist_csv_path}")
    
    print("\nDisplaying Processed Image:")
    display(IPImage(filename=img_path))

    print("\n--- Download Links --- ")
    try:
        files.download(img_path)
        files.download(csv_path)
        files.download(dist_csv_path)
    except Exception as e:
        print(f"Could not automatically download files. You can download them from the 'Processed_Output' folder in the file browser on the left. Error: {e}")
    return csv_path, dist_csv_path, img_path

# --- Main Execution Block --- 

if 'uploaded_image_path' in locals() and uploaded_image_path and os.path.exists(uploaded_image_path):
    image_to_process = uploaded_image_path
    print(f"Starting processing for: {image_to_process}")
    
    # 1. Find Elephants (Detection)
    txt_path, _ = Find_Elephants(image_to_process)

    if txt_path:
        # 2. Load Detections
        boxes = load_bounding_boxes_from_txt(txt_path)

        if boxes:
            # 3. Run GPS Pipeline
            gps_results = run_pipeline(image_to_process, boxes)

            if gps_results:
                # 4. Save and Display Results
                base_name = Path(image_to_process).stem
                save_and_display_results(image_to_process, gps_results, base_name)
            else:
                print("Could not calculate GPS coordinates.")
        else:
            print("No bounding boxes loaded. Cannot proceed with GPS calculation.")
    else:
        print("Detection failed. Cannot proceed.")
else:
    print("ERROR: No image was uploaded or found. Please run the 'Upload Image' cell first.")

--- 
## 4. (Optional) View Coordinates & Distances

In [ ]:
import pandas as pd

base_name = None
if 'uploaded_image_path' in locals() and uploaded_image_path:
    base_name = Path(uploaded_image_path).stem

if base_name:
    coords_file = f"Processed_Output/{base_name}_coordinates.csv"
    dists_file = f"Processed_Output/{base_name}_distances.csv"

    if os.path.exists(coords_file):
        print("--- Coordinates (First 10 rows) ---")
        df_coords = pd.read_csv(coords_file)
        print(df_coords.head(10).to_string())
    else:
        print(f"Coordinates file ({coords_file}) not found.")

    if os.path.exists(dists_file):
        print("\n--- Distances (First 10 rows) ---")
        df_dists = pd.read_csv(dists_file)
        print(df_dists.head(10).to_string())
    else:
        print(f"Distances file ({dists_file}) not found.")
else:
    print("Processing hasn't been run yet, or no image was uploaded.")